In [2]:
from pathlib import Path
from optimum.onnxruntime import (
    AutoQuantizationConfig,
    ORTModelForSpeechSeq2Seq,
    ORTQuantizer
)

# Configure base model and save directory for compressed model
model_id = "openai/whisper-large-v2"
save_dir = "whisper-large"

# Export model in ONNX
model = ORTModelForSpeechSeq2Seq.from_pretrained(model_id, export=True)
model_dir = model.model_save_dir

# Run quantization for all ONNX files of exported model
onnx_models = list(Path(model_dir).glob("*.onnx"))
print(onnx_models)
quantizers = [ORTQuantizer.from_pretrained(model_dir, file_name=onnx_model) for onnx_model in onnx_models]

qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

for quantizer in quantizers:
    # Apply dynamic quantization and save the resulting model
    quantizer.quantize(save_dir=save_dir, quantization_config=qconfig)

ModuleNotFoundError: No module named 'optimum'

In [1]:
import re
import whisper

## ground_truth 생성

In [8]:
import json

# 1️⃣ JSON 파일 로드
json_path = "dataset/Sample/02.라벨링데이터/서울/구급/651e464d69a4f266f06268a0_20220101.json"  # JSON 파일 경로
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2️⃣ "text" 값만 추출하여 `ground_truth` 리스트 생성
ground_truth = [utterance["text"] for utterance in data["utterances"]]

# 3️⃣ 결과 출력
print("\n✅ 추출된 Ground Truth 리스트:")
for text in ground_truth:
    print(text)


✅ 추출된 Ground Truth 리스트:
119니다.
어 네, 여기 저기 어디지? 고속더미널.
네.
그 맞은편 육교에 있는 육교 다리 밑인데요.
선생님, 그렇게 말하면 저희가 악기, 알기가 어려우니까 고속터미널에 해서 그 육교가 어느 방향 쪽이예요? 거기 근처에 주소 아세요?
네.
네.
네.
아, 고속터미널, 그 지하철역 3출구 앞에 있는 육교거든요.
3번 출구요?
네, 네, 네.
고속터미널 3호선 3번 출구?
네네네. 그.
아, 7호선 있구나, 그쪽이.
예, 7호선 3호선 다 다녀요, 그니까 어쨌든 3번 출구 7호선일 거예요, 아마.
네, 7호선 3호 출구, 그 옆에 스타벅스 있고, 그런데 쪽 말하는 거죠?
네.
네, 맞아요, 맞아요, 맞아요.
예, 그쪽 앞에 육교 쪽에, 네.
거기 육교, 다리 밑인데 여기 남성 한 분이 쓰러져 계시는데요.
네, 그, 한번.
그 경련도 좀 하시는 거 같고.
경련을 하세요?
예, 의식은 있으신데, 네.
의식은 있고 호흡도 있고 그분이 막 부들 떨고 있어요? 바들바들?
네, 호흡은 있는데.
네네, 몸을 떨고 계시구요. 지금 약간 배설물도 나오시는 거 같거든요
예, 그래요? 선생님, 그쪽 근처에 한번 가보시고.
네, 네.
네.
저희가 의료지도도 해드릴 거니깐 전화는 끊지 말구요. 잠시만요.
네, 네.
지금 구급차는 나갔어요. 저희가 해드릴 거니까 잠시만 기다려주세요.
네.
네.
네.


In [9]:
import whisper

# 1️⃣ 모델 로드 (small, medium, large 선택 가능)
model = whisper.load_model("large")  # "small", "medium", "large" 중 선택 가능

# 2️⃣ 음성 파일 변환
audio_path = "dataset/Sample/01.원천데이터/서울/구급/651e464d69a4f266f06268a0_20220101.wav"  # 변환할 오디오 파일 경로
result = model.transcribe(audio_path)

# 3️⃣ 변환된 텍스트 출력
print("\n✅ Whisper 변환 결과:")
print(result["text"])

/opt/anaconda3/envs/whisper/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Whisper 변환 결과:
 119입니다. 어 네 여기 저기 어디지? 고속터미널 맞은편 6교에 있는 6교 발입 위치인데요. 네 그렇게 말하면 저희가 알기가 어려우니까 고속터미널에서 그 6교가 어느 방향 쪽이에요? 거기 근처에 주소 아세요? 아 고속터미널 그 지하철역 3번 출구 앞에 있는 6교거든요. 3번 출구요? 네 네 네. 고속터미널 3호선 3번 출구? 아 7호선이구나 그쪽이. 네 7호선 3호선 다 다녀요. 그니까 어쨌든 3번 출구 7호선일 거예요 아마. 네 7호선 3번 출구 그 옆에 스타벅스 있고 그런 데 쪽 말하는 거죠? 네 맞아요 맞아요 맞아요. 네 그쪽 아시겠죠? 네 네. 발이 밑인데 여기 남동 1분이 쓰러져 계시는데요. 네 그 1번. 정렴도 좀 하시는 거 같고. 정리 안 해도 돼요? 네 의식은 있으신데 네. 의식은 있고 호흡도 있고 그분이 막 끌고 있어요? 네 네 몸을 떨고 계시구요. 지금 약간 배설물도 나오시는 거 같거든요. 네 그래요? 선생님 그쪽 근처로 1번 가 보시고 저희가 의료지도도 해드릴 거니까 전화는 끊지 말고요. 잠시만요. 네 네. 지금 구급차는 나갔어요. 저희가 드릴 거니까 잠시만 기다려주세요.


# 텍스트 전처리

In [24]:
import re
from jiwer import wer

def preprocess_text(text):
    """WER 계산을 위해 ground_truth와 whisper_text를 동일한 형식으로 변환하는 함수"""
    text = text.lower()  # 대소문자 통일
    text = text.replace("\n", " ")  # 줄바꿈을 띄어쓰기로 변경
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
    return text

In [21]:
whisper_text = result["text"]
ground_truth_str = " ".join(ground_truth)


AttributeError: 'str' object has no attribute 'dtypes'

In [25]:
ground_truth_cleaned = preprocess_text(ground_truth_str)
whisper_text_cleaned = preprocess_text(whisper_text)

In [27]:
whisper_text_cleaned

'119입니다 어 네 여기 저기 어디지 고속터미널 맞은편 6교에 있는 6교 발입 위치인데요 네 그렇게 말하면 저희가 알기가 어려우니까 고속터미널에서 그 6교가 어느 방향 쪽이에요 거기 근처에 주소 아세요 아 고속터미널 그 지하철역 3번 출구 앞에 있는 6교거든요 3번 출구요 네 네 네 고속터미널 3호선 3번 출구 아 7호선이구나 그쪽이 네 7호선 3호선 다 다녀요 그니까 어쨌든 3번 출구 7호선일 거예요 아마 네 7호선 3번 출구 그 옆에 스타벅스 있고 그런 데 쪽 말하는 거죠 네 맞아요 맞아요 맞아요 네 그쪽 아시겠죠 네 네 발이 밑인데 여기 남동 1분이 쓰러져 계시는데요 네 그 1번 정렴도 좀 하시는 거 같고 정리 안 해도 돼요 네 의식은 있으신데 네 의식은 있고 호흡도 있고 그분이 막 끌고 있어요 네 네 몸을 떨고 계시구요 지금 약간 배설물도 나오시는 거 같거든요 네 그래요 선생님 그쪽 근처로 1번 가 보시고 저희가 의료지도도 해드릴 거니까 전화는 끊지 말고요 잠시만요 네 네 지금 구급차는 나갔어요 저희가 드릴 거니까 잠시만 기다려주세요'

In [28]:
wer_score =wer(ground_truth_cleaned, whisper_text_cleaned)
print(wer_score)
print(f"\n WER (Word Error Rate): {wer_score:.2%}")

0.3815028901734104

 WER (Word Error Rate): 38.15%


In [14]:
whisper_text

' 119입니다. 어 네 여기 저기 어디지? 고속터미널 맞은편 6교에 있는 6교 발입 위치인데요. 네 그렇게 말하면 저희가 알기가 어려우니까 고속터미널에서 그 6교가 어느 방향 쪽이에요? 거기 근처에 주소 아세요? 아 고속터미널 그 지하철역 3번 출구 앞에 있는 6교거든요. 3번 출구요? 네 네 네. 고속터미널 3호선 3번 출구? 아 7호선이구나 그쪽이. 네 7호선 3호선 다 다녀요. 그니까 어쨌든 3번 출구 7호선일 거예요 아마. 네 7호선 3번 출구 그 옆에 스타벅스 있고 그런 데 쪽 말하는 거죠? 네 맞아요 맞아요 맞아요. 네 그쪽 아시겠죠? 네 네. 발이 밑인데 여기 남동 1분이 쓰러져 계시는데요. 네 그 1번. 정렴도 좀 하시는 거 같고. 정리 안 해도 돼요? 네 의식은 있으신데 네. 의식은 있고 호흡도 있고 그분이 막 끌고 있어요? 네 네 몸을 떨고 계시구요. 지금 약간 배설물도 나오시는 거 같거든요. 네 그래요? 선생님 그쪽 근처로 1번 가 보시고 저희가 의료지도도 해드릴 거니까 전화는 끊지 말고요. 잠시만요. 네 네. 지금 구급차는 나갔어요. 저희가 드릴 거니까 잠시만 기다려주세요.'

## faster whisper model - large by quantization

In [1]:
#(7-1) Make user-defined functions for STT
def split_into_sentences(text):
    sentences = re.split(r'(?<=[?.])\s*', text)
    return sentences

def extract_text_from_file(file_path: str) -> dict :
    model_size = "large-v3"
    model = WhisperModel(model_size, device='auto', compute_type = "int8")
    segments, info = model.transcribe(
        file_path,
        language="ko",
        beam_size=10,
        word_timestamps=True,
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=500)
    )
    return segments

## whisper model - quantization

In [29]:
import whisper
import torch

# ✅ 1️⃣ 모델 로드
model = whisper.load_model("large")

# ✅ 2️⃣ Quantization 적용 (INT8)
model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# ✅ 3️⃣ 음성 파일 변환
audio_path = "dataset/Sample/01.원천데이터/서울/구급/651e464d69a4f266f06268a0_20220101.wav"
result = model.transcribe(audio_path)

# ✅ 4️⃣ 변환된 텍스트 출력
print("\n✅ Whisper 변환 결과 (INT8 적용):")
print(result["text"])

/opt/anaconda3/envs/whisper/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Whisper 변환 결과 (INT8 적용):
 119입니다. 어 네 여기 저기 어디지? 고속터미널 맞은편 6교에 있는 6교 발입 위치인데요. 네 그렇게 말하면 저희가 알기가 어려우니까 고속터미널에서 그 6교가 어느 방향 쪽이에요? 거기 근처에 주소 아세요? 아 고속터미널 그 지하철역 3번 출구 앞에 있는 6교거든요. 3번 출구요? 네 네 네. 고속터미널 3호선 3번 출구? 아 7호선이구나 그쪽이. 네 7호선 3호선 다 다녀요. 그니까 어쨌든 3번 출구 7호선일 거예요 아마. 네 7호선 3번 출구 그 옆에 스타벅스 있고 그런 데 쪽 말하는 거죠? 네 맞아요 맞아요 맞아요. 네 그쪽 아시겠죠? 네 네. 발이 밑인데 여기 남동 1분이 쓰러져 계시는데요. 네 그 1번. 정렴도 좀 하시는 거 같고. 정리 안 해도 돼요? 네 의식은 있으신데 네. 의식은 있고 호흡도 있고 그분이 막 끌고 있어요? 네 네 몸을 떨고 계시구요. 지금 약간 배설물도 나오시는 거 같거든요. 네 그래요? 선생님 그쪽 근처로 1번 가 보시고 저희가 의료지도도 해드릴 거니까 전화는 끊지 말고요. 잠시만요. 네 네. 지금 구급차는 나갔어요. 저희가 드릴 거니까 잠시만 기다려주세요.


In [33]:
model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [38]:
import whisper
import torch
import onnxruntime as ort

# ✅ Whisper 모델 로드 (기본 FP32)
model = whisper.load_model("large")

# ✅ ONNX로 변환하여 CPU에서 최적화 실행
dummy_input = torch.randn(1, 80, 3000)
torch.onnx.export(model, dummy_input, "whisper_model.onnx")

# ✅ ONNX 모델 실행
session = ort.InferenceSession("whisper_model.onnx")
audio_path = "dataset/Sample/01.원천데이터/서울/구급/651e464d69a4f266f06268a0_20220101.wav"
result = model.transcribe(audio_path)

# ✅ 변환된 텍스트 출력
print("\n✅ Whisper 변환 결과 (ONNX 최적화 적용):")
print(result["text"])

✅ 현재 사용 중인 디바이스: mps


NotImplementedError: Could not run 'aten::_sparse_coo_tensor_with_dims_and_tensors' with arguments from the 'SparseMPS' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_sparse_coo_tensor_with_dims_and_tensors' is only available for these backends: [MPS, Meta, SparseCPU, SparseMeta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedT

✅ 현재 사용 중인 디바이스: mps
